In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import math,time,os

In [2]:
def parse_label(path):
    raw = open(path).read()
    classnames =[]
    for cls in raw.split('\n'):
        if cls == '':
            continue
        classnames.append(cls)
    return classnames

def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)  # 读取原始文件
    image_decoded = tf.io.decode_jpeg(image_string,channels=3)  # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [300, 300]) / 255.0
    return image_resized, label

def load_data(dirs):
    images_name = []
    labels = []
    for i,file_dir in enumerate(dirs):
        filename = tf.constant([file_dir + '/' + filename for filename in os.listdir(file_dir)])
        images_name = tf.concat([images_name,filename],axis=-1)
        labels = tf.concat([labels,tf.constant(i,shape=filename.shape[0])],axis=-1)
    print("total:%d" %  images_name.shape[0])
    train_dataset = tf.data.Dataset.from_tensor_slices((images_name, labels))
    train_dataset = train_dataset.map(
        map_func=_decode_and_resize,
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return train_dataset

num_epochs = 1
batch_size = 5
learning_rate = 0.01
buffer_size=10000

train_dir = './data/train/'
class_names = parse_label(train_dir+'classes_label.txt')
train_file_dirs = [train_dir+classname for classname in class_names] 
train_dataset = load_data(train_file_dirs)
train_dataset = train_dataset.shuffle(buffer_size)
train_dataset = train_dataset.batch(batch)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

total:94175


In [ ]:
model = tf.keras.applications.EfficientNetB3(weights=None,classes=len(class_names))
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )
model.fit(train_dataset,epochs=num_epochs)

12163/18835 [==================>...........] - ETA: 54:38 - loss: 2.2312 - sparse_categorical_accuracy: 0.2553 ETA: 52:41 - loss: 2.2311 - sparse_categorical_